In [2]:
from PIL import Image
import random
import numpy as np
import os

def load_image(image_path):
    return Image.open(image_path).convert('RGB')
    
def  getRGBMeanPixel(image):
    #Returns average RGB color of image
    imageArr = np.array(image) #image as numpy array
    
    width, height, dimension = imageArr.shape

    imageArr.shape=(width*height, dimension) #reshape array

    return tuple(imageArr.mean(axis=0)) #return image mean

def getRGBMeanImage(image_path, folder): # image path and array of image labels

    totalMean = [0,0,0]
    #images = len(folder)

    for images in folder:
        img = Image.open(os.path.join(image_path, images)).convert('RGB')
        pixelMean = getRGBMeanPixel(img)
        
        for j in range(3):
            totalMean[j] = pixelMean[j] + totalMean[j]

    for i in range(3):
        totalMean[i] = totalMean[i]/len(folder)

    return tuple(totalMean)

def image_preprocess(img):
    width = img.size[0]
    height = img.size[1]
    Asp_ratio = width/height
    if width < height:
        new_h = int(256 * (1/Asp_ratio))
        img = img.resize((256, new_h), Image.BICUBIC)
    else:
        new_w = int(256 * Asp_ratio)
        img = img.resize((new_w, 256), Image.BICUBIC)
        
    return img

def getpatches(image_path, n):
    img = load_image(image_path)
    img =  image_preprocess(img)
    patches=[]
    IMAGENET_MEAN = [123.68, 116.779, 103.939]
    for i in range(n):
        x = random.randint(0, img.size[0] - 227)
        y = random.randint(0, img.size[1] - 227)
        img_cropped = img.crop((x, y, x + 227, y + 227))

        cropped_im_array = np.array(img_cropped, dtype=np.float32)
        
        #hor_flip_array = np.array(img_cropped.transpose(Image.FLIP_LEFT_RIGHT), dtype=np.float32) 
        
    
        for i in range(3):
            cropped_im_array[:,:,i] -= IMAGENET_MEAN[i]
            #hor_flip_array[:,:,i] -= IMAGENET_MEAN[i]
            
        patches.append(cropped_im_array)
        #patches.append(hor_flip_array)
        
    np.vstack(patches)
    
    return patches

def read_gt_label(file_path, classes):
    #Reads the groundtruth label from ILSVRX2012 gt validation file

    labels = []

    file = open(file_path)

    for line in file:
        one = np.zeros(classes)
        value = int(line) - 1
        one[value]= 1.0
        labels.append(one)

    file.close()
    np.vstack(labels)

    return labels


def convLayer(x, strideX, strideY, weight, bias, padding = "SAME", groups = 1):
    ch = int(x.get_shape()[-1])
    conv = lambda a, b: tf.nn.conv2d(a, b, strides = [1, strideY, strideX, 1], padding = padding)
    inp_spl = tf.split(value = x, num_or_size_splits = groups, axis = 3)
    weight_spl = tf.split(value = weight, num_or_size_splits = groups, axis = 3)
    fm = [conv(t1, t2) for t1, t2 in zip(inp_spl, weight_spl)]
    comb_fm = tf.concat(axis = 3, values = fm)
    output = tf.nn.bias_add(comb_fm, bias)
    return output

def flatten(inp):
    shape = inp.get_shape().as_list()        
    dim = np.prod(shape[1:])            
    flat = tf.reshape(inp, [-1, dim])
    
    return flat

def loss_function(labels, output):
    return tf.reduce_mean(-tf.reduce_sum(tf.log(output+1e-4)))

def read_gt_label_train(file_path, classes):
    
    labels = []

    file = open(file_path)

    for line in file:
        one = np.zeros(classes)
        value = int(line)
        one[value]= 1.0
        labels.append(one)

    file.close()
    np.vstack(labels)

    return labels



  


    
    




